# LightGBM

# Model

In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import BaggingRegressor

In [2]:
hit = pd.read_csv("Hitters.csv")
df = hit.copy()
df = df.dropna()
dms = pd.get_dummies(df[['League', 'Division', 'NewLeague']])
y = df["Salary"]
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')
X = pd.concat([X_, dms[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25, 
                                                    random_state=42)

In [3]:
#!pip install lightgbm , error verebiliyor

In [4]:
from lightgbm import LGBMRegressor

In [7]:
#conda install -c conda-forge lightgbm , üstteki eror verince bununla indirdim çözüldü

In [31]:
lgbm = LGBMRegressor(random_state = 42)
lgbm_model = lgbm.fit(X_train, y_train)

# Tahmin

In [32]:
y_pred = lgbm_model.predict(X_test, 
                            num_iteration = lgbm_model.best_iteration_)

In [33]:
np.sqrt(mean_squared_error(y_test, y_pred))

363.8712087611089

[CV] END colsample_bytree=1, learning_rate=0.1, max_depth=7, n_estimators=100; total time=   0.0s
[CV] END colsample_bytree=1, learning_rate=0.1, max_depth=7, n_estimators=100; total time=   0.0s
[CV] END colsample_bytree=1, learning_rate=0.1, max_depth=7, n_estimators=100; total time=   0.0s
[CV] END colsample_bytree=1, learning_rate=0.1, max_depth=7, n_estimators=100; total time=   0.0s
[CV] END colsample_bytree=1, learning_rate=0.1, max_depth=7, n_estimators=100; total time=   0.0s
[CV] END colsample_bytree=1, learning_rate=0.1, max_depth=7, n_estimators=100; total time=   0.0s
[CV] END colsample_bytree=1, learning_rate=0.1, max_depth=7, n_estimators=200; total time=   0.0s
[CV] END colsample_bytree=1, learning_rate=0.1, max_depth=7, n_estimators=200; total time=   0.0s
[CV] END colsample_bytree=1, learning_rate=0.1, max_depth=8, n_estimators=20; total time=   0.0s
[CV] END colsample_bytree=1, learning_rate=0.1, max_depth=8, n_estimators=20; total time=   0.0s
[CV] END colsample_byt

# Model Tuning

In [11]:
lgbm_model.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': 'warn',
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [12]:
lgbm_grid = {
    'colsample_bytree': [0.4, 0.5,0.6,0.9,1],
    'learning_rate': [0.01, 0.1, 0.5,1],
    'n_estimators': [20, 40, 100, 200, 500,1000],
    'max_depth': [1,2,3,4,5,6,7,8] }

lgbm = LGBMRegressor()
lgbm_cv_model = GridSearchCV(lgbm, lgbm_grid, cv=10, n_jobs = -1, verbose = 2)

In [13]:
lgbm_cv_model.fit(X_train, y_train)

Fitting 10 folds for each of 960 candidates, totalling 9600 fits
[CV] END colsample_bytree=0.4, learning_rate=0.01, max_depth=1, n_estimators=20; total time=   0.0s
[CV] END colsample_bytree=0.4, learning_rate=0.01, max_depth=1, n_estimators=40; total time=   0.0s
[CV] END colsample_bytree=0.4, learning_rate=0.01, max_depth=1, n_estimators=100; total time=   0.0s
[CV] END colsample_bytree=0.4, learning_rate=0.01, max_depth=1, n_estimators=100; total time=   0.0s
[CV] END colsample_bytree=0.4, learning_rate=0.01, max_depth=1, n_estimators=200; total time=   0.0s
[CV] END colsample_bytree=0.4, learning_rate=0.01, max_depth=1, n_estimators=500; total time=   0.0s
[CV] END colsample_bytree=0.4, learning_rate=0.01, max_depth=1, n_estimators=500; total time=   0.0s
[CV] END colsample_bytree=0.4, learning_rate=0.01, max_depth=1, n_estimators=1000; total time=   0.0s
[CV] END colsample_bytree=0.4, learning_rate=0.01, max_depth=1, n_estimators=1000; total time=   0.1s
[CV] END colsample_bytree=

GridSearchCV(cv=10, estimator=LGBMRegressor(), n_jobs=-1,
             param_grid={'colsample_bytree': [0.4, 0.5, 0.6, 0.9, 1],
                         'learning_rate': [0.01, 0.1, 0.5, 1],
                         'max_depth': [1, 2, 3, 4, 5, 6, 7, 8],
                         'n_estimators': [20, 40, 100, 200, 500, 1000]},
             verbose=2)

In [14]:
lgbm_cv_model.best_params_

{'colsample_bytree': 0.5,
 'learning_rate': 0.1,
 'max_depth': 6,
 'n_estimators': 20}

In [17]:
# final model

In [26]:
lgbm_tuned = LGBMRegressor(learning_rate = 0.1, 
                           max_depth = 6, 
                           n_estimators = 20,
                          colsample_bytree = 0.5,
                          random_state = 42)

lgbm_tuned = lgbm_tuned.fit(X_train,y_train)

In [27]:
y_pred = lgbm_tuned.predict(X_test)

In [28]:
np.sqrt(mean_squared_error(y_test, y_pred))

375.2220482868922

# Category Boosting (CatBoost)

# Model

In [34]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import BaggingRegressor

In [35]:
hit = pd.read_csv("Hitters.csv")
df = hit.copy()
df = df.dropna()
dms = pd.get_dummies(df[['League', 'Division', 'NewLeague']])
y = df["Salary"]
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')
X = pd.concat([X_, dms[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25, 
                                                    random_state=42)

In [36]:
!pip install catboost

     |████████████████████████████████| 22.0 MB 13.1 MB/s eta 0:00:01
     |████████████████████████████████| 47 kB 17.5 MB/s eta 0:00:01
     |████████████████████████████████| 15.2 MB 8.3 MB/s eta 0:00:011


In [37]:
from catboost import CatBoostRegressor

In [38]:
catb = CatBoostRegressor()
catb_model = catb.fit(X_train, y_train)

Learning rate set to 0.031674
0:	learn: 437.6430699	total: 59.7ms	remaining: 59.6s
1:	learn: 431.3923642	total: 60.6ms	remaining: 30.3s
2:	learn: 424.8820360	total: 61.6ms	remaining: 20.5s
3:	learn: 418.2514904	total: 62.6ms	remaining: 15.6s
4:	learn: 412.6394021	total: 63.7ms	remaining: 12.7s
5:	learn: 406.6247020	total: 64.7ms	remaining: 10.7s
6:	learn: 400.5321206	total: 65.7ms	remaining: 9.31s
7:	learn: 394.6683437	total: 66.6ms	remaining: 8.26s
8:	learn: 388.2496484	total: 67.7ms	remaining: 7.46s
9:	learn: 382.9448842	total: 68.9ms	remaining: 6.82s
10:	learn: 377.2600080	total: 70.6ms	remaining: 6.34s
11:	learn: 372.4829606	total: 71.6ms	remaining: 5.89s
12:	learn: 366.6823437	total: 72.6ms	remaining: 5.51s
13:	learn: 362.6076230	total: 73.6ms	remaining: 5.18s
14:	learn: 358.0107745	total: 74.8ms	remaining: 4.91s
15:	learn: 353.2802665	total: 75.9ms	remaining: 4.67s
16:	learn: 348.5646265	total: 77ms	remaining: 4.45s
17:	learn: 343.6407912	total: 78.3ms	remaining: 4.27s
18:	learn:

# Tahmin

In [40]:
y_pred = catb_model.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

351.194631344607

# Model Tuning

In [41]:
catb_grid = {
    'iterations': [200,500,1000,2000],
    'learning_rate': [0.01, 0.03, 0.05, 0.1],
    'depth': [3,4,5,6,7,8] }

In [42]:
catb = CatBoostRegressor()
catb_cv_model = GridSearchCV(catb, catb_grid, cv=5, n_jobs = -1, verbose = 2)

In [45]:
#catb_cv_model.fit(X_train, y_train)

In [46]:
# yukarıdaki kodla cv yapıyoruz modele, ama catboost çok uzun sürdüğü için yapmadım

In [47]:
catb_cv_model.best_params_

{'depth': 5, 'iterations': 1000, 'learning_rate': 0.1}

In [50]:
catb_tuned = CatBoostRegressor(iterations = 1000, 
                               learning_rate = 0.1, 
                               depth = 5)

catb_tuned = catb_tuned.fit(X_train,y_train)

0:	learn: 422.4143448	total: 1.09ms	remaining: 1.09s
1:	learn: 404.1864276	total: 2.07ms	remaining: 1.03s
2:	learn: 386.3231718	total: 2.8ms	remaining: 932ms
3:	learn: 370.5548032	total: 3.75ms	remaining: 935ms
4:	learn: 354.9242038	total: 4.73ms	remaining: 942ms
5:	learn: 342.3403984	total: 5.9ms	remaining: 978ms
6:	learn: 328.2370070	total: 6.75ms	remaining: 958ms
7:	learn: 317.5056526	total: 7.46ms	remaining: 925ms
8:	learn: 306.6243511	total: 8.52ms	remaining: 938ms
9:	learn: 297.3147023	total: 9.4ms	remaining: 931ms
10:	learn: 288.3685892	total: 10.4ms	remaining: 938ms
11:	learn: 281.0996220	total: 11.7ms	remaining: 962ms
12:	learn: 273.2254898	total: 12.5ms	remaining: 948ms
13:	learn: 266.9003385	total: 13.2ms	remaining: 930ms
14:	learn: 261.9092500	total: 14.2ms	remaining: 933ms
15:	learn: 256.2637350	total: 15.4ms	remaining: 949ms
16:	learn: 250.3667935	total: 16.3ms	remaining: 942ms
17:	learn: 244.8631098	total: 17ms	remaining: 926ms
18:	learn: 240.1540669	total: 17.8ms	remain

In [51]:
y_pred = catb_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

356.665762904938